In [1]:
from pyannote.audio import Pipeline
import os
from dotenv import load_dotenv

load_dotenv()

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=huggingface_token)

C:\Users\verho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


In [ ]:
import torch
# pipeline.to(torch.device("cuda"))

# apply pretrained pipeline
diarization = pipeline("../training_database/wavs/file1.wav")

# print the result
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
# start=0.2s stop=1.5s speaker_0
# start=1.8s stop=3.9s speaker_1
# start=4.2s stop=5.7s speaker_0
# ...

In [9]:
print(len(diarization))

135


In [ ]:
tracks = []
for segment in diarization.itersegments():
    print(type(segment))
    track = diarization.get_tracks(segment)
    tracks.append(track)
    print(type(next(iter(track))))

In [14]:
from pydub import AudioSegment

audio = AudioSegment.from_wav("../training_database/wavs/file1.wav")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    segment = audio[turn.start * 1000:turn.end * 1000]  # Convert seconds to milliseconds
    segment.export(f"./output/pyannote/speaker_{speaker}_{turn.start:.1f}-{turn.end:.1f}.wav", format="wav")
